<a href="https://colab.research.google.com/github/DIPANJAN001/Andrew-Ng-Machine-Learning-Notes/blob/master/fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Define the hyperparameter search space
def build_model(hp):
    text_input = Input(shape=(max_text_sequence_length,))
    text_embedding_layer = Embedding(input_dim=max_text_words, output_dim=64)(text_input)
    text_lstm = LSTM(64)(text_embedding_layer)

    categorical_input = Input(shape=(X_categorical_train.shape[1],))
    categorical_embedding_layer = Embedding(input_dim=np.max(X_categorical_train) + 1, output_dim=32)(categorical_input)
    categorical_flatten = Flatten()(categorical_embedding_layer)

    merged = Concatenate()([text_lstm, categorical_flatten])

    dense_layer = Dense(hp.Int('dense_units', min_value=32, max_value=256, step=32), activation='relu')(merged)
    output_layer = Dense(1, activation='sigmoid')(dense_layer)

    model = Model(inputs=[text_input, categorical_input], outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), metrics=['accuracy'])

    return model

tuner = RandomSearch(
   build_model,
   objective='val_accuracy',
   max_trials=10,
   executions_per_trial=1,
   directory='my_tuner_directory',
   project_name='siamese_claim_severity'
)

# Perform the hyperparameter search, passing class_weight_dict to model.fit
tuner.search(
   [X_text_train, X_categorical_train],
   y_train,
   validation_data=([X_text_test, X_categorical_test], y_test),
   epochs=10,
   class_weight=class_weight_dict  # Include class weights here
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build and train the final model with the best hyperparameters
best_model = build_model(best_hps)
best_model.fit(
   [X_text_train, X_categorical_train],
   y_train,
   batch_size=32,
   epochs=10,
   validation_split=0.1,
   class_weight=class_weight_dict  # Include class weights here as well
)